In [1]:
import json

import altair as alt
import polars as pl
from fit2parquets.parser import Parser

In [2]:
# https://gist.githubusercontent.com/thomascamminady/c5da0b7acb41faf6abd6c99aff10e144/raw/f2b074e2e3945c0102eceb4983eb998da1432d44/theme.json
@alt.theme.register("custom_theme", enable=True)
def custom_theme():
    with open("theme.json", "r") as f:
        return json.load(f)


alt.data_transformers.disable_max_rows()
alt.renderers.enable("browser")
# alt.renderers.set_embed_options(actions=False)

RendererRegistry.enable('browser')

In [14]:
Parser().fit2parquets("../TRACKR_Fenix7Pro_Race.fit")
df = (
    pl.read_parquet("../TRACKR_Fenix7Pro_Race/record_mesgs.parquet").unpivot(
        index="timestamp", value_name="value", variable_name="key"
    )
    # .with_columns(
    #     (
    #         (pl.col("timestamp") - pl.col("timestamp").min()).dt.total_seconds()
    #     ).alias("timestamp")
    # )
    # .unpivot(index="timestamp")
)

In [15]:
brush = alt.selection_interval(encodings=["x"])
base = alt.Chart(df).add_params(brush).properties(width=800, height=200)

linecolor = "#1f77b4"
rulecolor = "firebrick"
chart = (
    alt.layer(
        base.mark_line(color=linecolor).encode(
            x=alt.X("timestamp:T").title("Elapsed time (seconds)"),
            y=alt.Y("value:Q").scale(zero=False).title(""),
            opacity=alt.value(0.2),
        ),
        base.transform_filter(brush)
        .mark_line(color=linecolor)
        .encode(
            x=alt.X("timestamp:T"),
            y=alt.Y("value:Q").scale(zero=False),
            opacity=alt.value(1),
        ),
        base.transform_filter(brush)
        .mark_rule(color=rulecolor, strokeDash=[5, 5])
        .encode(
            x=alt.X("min(timestamp):T"),
            x2=alt.X2("max(timestamp):T"),
            y=alt.Y("min(value):Q").scale(zero=False),
        ),
        base.transform_filter(brush)
        .mark_text(color=linecolor, dy=-50, dx=0, size=20)
        .encode(
            x=alt.X("mean(timestamp):T"),
            y=alt.Y("min(value):Q").scale(zero=False),
            text=alt.Text("mean(value):Q", format=".3f"),
        ),
        base.transform_filter(brush)
        .mark_text(color=rulecolor, dx=0, size=14, dy=-10)
        .encode(
            x=alt.X("mean(timestamp):T"),
            y=alt.Y("min(value):Q").scale(zero=False),
            text=alt.Text("count(value):Q", format=".3f"),
        ),
        base.transform_filter(brush)
        .mark_text(color=rulecolor, dx=-20, size=14, dy=-10)
        .encode(
            x=alt.X("min(timestamp):T"),
            y=alt.Y("min(value):Q").scale(zero=False),
            text=alt.Text(
                "min(timestamp):T",
                formatType="time",
                timeUnit="hoursminutesseconds",
            ),
        ),
        base.transform_filter(brush)
        .mark_text(color=rulecolor, dx=20, size=14, dy=-10)
        .encode(
            x=alt.X("max(timestamp):T"),
            y=alt.Y("min(value):Q").scale(zero=False),
            text=alt.Text(
                "max(timestamp):T",
                formatType="time",
                timeUnit="hoursminutesseconds",
            ),
        ),
    )
    .facet(
        row=alt.Row("key:N").header(labelOrient="right", labelAngle=0),
        spacing=25,
    )
    .resolve_scale(y="independent", x="independent")
)
with open("chart.json", "w") as f:
    d = chart.to_dict()
    # del d["data"]
    del d["datasets"]
    f.write(json.dumps(d, indent=2))
chart.show()